In [9]:
from bloqade import squin
from typing import Dict, Tuple, Optional, Callable
from kirin.dialects.ilist import IList
import numpy as np
import bloqade.stim
import bloqade.tsim

In [13]:
POI_CONFIG: Dict[str, Tuple[str, float]] = {}

def configure_poi(config: Dict[str, Tuple[str, float]]):
    global POI_CONFIG
    POI_CONFIG = config.copy()
    poi_kernels = dict()
    for x in POI_CONFIG.keys():
        poi_kernels[x] = get_poi_kernel(x)
    return poi_kernels
    

def get_poi_kernel(label: str):
    
    config = POI_CONFIG.get(label, ("none", 0.0))
    noise_type, prob = config
    
    if noise_type == "depolarize":
        @squin.kernel
        def _poi(q):
            squin.depolarize(prob, q)
        return _poi
    elif noise_type == "qubit_loss":
        @squin.kernel
        def _poi(q):
            squin.qubit_loss(prob, q)
        return _poi
    else:
        @squin.kernel
        def _poi(q):
            squin.u3(0, 0, 0, q)
        return _poi

In [14]:
# General MSD + Error Detection Circuit

def syndrome_extraction(theta, phi,lam, phys_qubits = 7, basis="z"):
    @squin.kernel
    # General MSD encoding with parameterized input state
    def parameterized_MSD_encoding(q_subset,theta,phi,lam):
        # Prepare injected state on last qubit
        squin.u3(theta,phi,lam,q_subset[6])

        # Apply MSD encoding circuit
        for i in range(6):
            squin.sqrt_y_adj(q_subset[i]) 
        squin.cz(q_subset[1], q_subset[2])
        squin.cz(q_subset[3], q_subset[4])
        squin.cz(q_subset[5], q_subset[6])
        squin.sqrt_y(q_subset[6])
        squin.cz(q_subset[0],q_subset[3])
        squin.cz(q_subset[2],q_subset[5])
        squin.cz(q_subset[4],q_subset[6])
        for i in range(5):
            squin.sqrt_y(q_subset[i+2])
        squin.cz(q_subset[0],q_subset[1])
        squin.cz(q_subset[2],q_subset[3])
        squin.cz(q_subset[4],q_subset[5])
        squin.sqrt_y(q_subset[1])
        squin.sqrt_y(q_subset[2])
        squin.sqrt_y(q_subset[4])
        
    @squin.kernel
    def circ():
        # Allocate 14 qubits: 7 for injected state, 7 for ancilla
        q=squin.qalloc(14)

        # Encode qubits
        parameterized_MSD_encoding([q[0],q[1],q[2],q[3],q[4],q[5],q[6]],theta=theta,phi=phi,lam=lam, noise_toggle=True) # Injected state encoding
        parameterized_MSD_encoding([q[7],q[8],q[9],q[10],q[11],q[12],q[13]],theta=np.pi/2,phi=0,lam=np.pi, noise_toggle=True) # Z-basis ancilla encoding

        # Entangle injected state with Z-basis ancilla (using CNOTs with injected state as control and ancilla as target)
        squin.cx(q[0],q[7])
        squin.cx(q[1],q[8])
        squin.cx(q[2],q[9])
        squin.cx(q[3],q[10])
        squin.cx(q[4],q[11])
        squin.cx(q[5],q[12])
        squin.cx(q[6],q[13])

        # Measure the ancilla qubits in the Z-basis
        z_measurement = squin.broadcast.measure([q[7],q[8],q[9],q[10],q[11],q[12],q[13]]) 

        # Reset the ancilla qubits to reuse them for X-basis ancilla
        squin.broadcast.reset([q[7], q[8], q[9], q[10], q[11], q[12], q[13]])

        # X-basis ancilla encoding
        parameterized_MSD_encoding([q[7],q[8],q[9],q[10],q[11],q[12],q[13]],theta=0,phi=0,lam=0, noise_toggle=True)

        # Entangle injected state with X-basis ancilla (using CNOTs with ancilla as control and injected state as target)
        squin.cx(q[7],q[0])
        squin.cx(q[8],q[1])
        squin.cx(q[9],q[2])
        squin.cx(q[10],q[3])
        squin.cx(q[11],q[4])
        squin.cx(q[12],q[5])
        squin.cx(q[13],q[6])

        # Measure the ancilla qubits in the X-basis (apply Hadamard before measurement)
        squin.broadcast.h([q[7], q[8], q[9], q[10], q[11], q[12], q[13]])
        x_measurement = squin.broadcast.measure([q[7],q[8],q[9],q[10],q[11],q[12],q[13]])
        
    return circ

In [15]:
MSD_enc = syndrome_extraction(1,0,0)
tsim_circ = bloqade.tsim.Circuit(MSD_enc)
tsim_circ.diagram(height=400)

In [18]:
# Configure Point(s) of Interest for Noise Injection (label: (noise_type, probability))
poi = configure_poi({"a": ("depolarize", 0.5)}) 

def syndrome_extraction(theta, phi,lam, phys_qubits = 7, basis="z"):
    # Call the appropriate Squin function for noise injection (based on POI's type and probability)
    poi_a = get_poi_kernel("a")

    @squin.kernel
    # General MSD encoding with parameterized input state
    def parameterized_MSD_encoding(q_subset,theta,phi,lam):
        # Prepare injected state on last qubit
        squin.u3(theta,phi,lam,q_subset[6])

        # Apply MSD encoding circuit
        for i in range(6):
            squin.sqrt_y_adj(q_subset[i]) 
        squin.cz(q_subset[1], q_subset[2])
        squin.cz(q_subset[3], q_subset[4])
        squin.cz(q_subset[5], q_subset[6])
        squin.sqrt_y(q_subset[6])
        squin.cz(q_subset[0],q_subset[3])
        squin.cz(q_subset[2],q_subset[5])
        squin.cz(q_subset[4],q_subset[6])
        for i in range(5):
            squin.sqrt_y(q_subset[i+2])
        squin.cz(q_subset[0],q_subset[1])
        squin.cz(q_subset[2],q_subset[3])
        squin.cz(q_subset[4],q_subset[5])
        squin.sqrt_y(q_subset[1])
        squin.sqrt_y(q_subset[2])
        squin.sqrt_y(q_subset[4])

        # Inject noise at this point in the circuit on all 7 logical qubits (for example)
        poi_a(q_subset[0]) # This will apply noise at this point for all calls to this kernel
        poi_a(q_subset[1])
        poi_a(q_subset[2])
        poi_a(q_subset[3])
        poi_a(q_subset[4])
        poi_a(q_subset[5])
        poi_a(q_subset[6])
        
    @squin.kernel
    def circ():
        # Allocate 14 qubits: 7 for injected state, 7 for ancilla
        q=squin.qalloc(14)

        # Encode qubits
        parameterized_MSD_encoding([q[0],q[1],q[2],q[3],q[4],q[5],q[6]],theta=theta,phi=phi,lam=lam, noise_toggle=True) # Injected state encoding
        parameterized_MSD_encoding([q[7],q[8],q[9],q[10],q[11],q[12],q[13]],theta=np.pi/2,phi=0,lam=np.pi, noise_toggle=True) # Z-basis ancilla encoding

        # Entangle injected state with Z-basis ancilla (using CNOTs with injected state as control and ancilla as target)
        squin.cx(q[0],q[7])
        squin.cx(q[1],q[8])
        squin.cx(q[2],q[9])
        squin.cx(q[3],q[10])
        squin.cx(q[4],q[11])
        squin.cx(q[5],q[12])
        squin.cx(q[6],q[13])

        # Measure the ancilla qubits in the Z-basis
        z_measurement = squin.broadcast.measure([q[7],q[8],q[9],q[10],q[11],q[12],q[13]]) 

        # Reset the ancilla qubits to reuse them for X-basis ancilla
        squin.broadcast.reset([q[7], q[8], q[9], q[10], q[11], q[12], q[13]])

        # X-basis ancilla encoding
        parameterized_MSD_encoding([q[7],q[8],q[9],q[10],q[11],q[12],q[13]],theta=0,phi=0,lam=0, noise_toggle=True)

        # Entangle injected state with X-basis ancilla (using CNOTs with ancilla as control and injected state as target)
        squin.cx(q[7],q[0])
        squin.cx(q[8],q[1])
        squin.cx(q[9],q[2])
        squin.cx(q[10],q[3])
        squin.cx(q[11],q[4])
        squin.cx(q[12],q[5])
        squin.cx(q[13],q[6])

        # Measure the ancilla qubits in the X-basis (apply Hadamard before measurement)
        squin.broadcast.h([q[7], q[8], q[9], q[10], q[11], q[12], q[13]])
        x_measurement = squin.broadcast.measure([q[7],q[8],q[9],q[10],q[11],q[12],q[13]])
        
    return circ

In [19]:
MSD_enc = syndrome_extraction(1,0,0)
tsim_circ = bloqade.tsim.Circuit(MSD_enc)
tsim_circ.diagram(height=400)